## 8. Local test
We can test our AI module on the IoT Edge Device. Since our AI module deployed to edge device and listening any requests from port 5001 (which is set in manifest file) we can send some test requests and see the results.

Also in a web browser on the IoT Edge device, we can visit the following URL addresses to see the state of our AI module.

```
http://127.0.0.1:5001/version

or

http://127.0.0.1:5001/about
```

### 8.1. Get global variables
We will read the previously stored variables. We need the name of the directory that we will use to store in our ml solution files. We create a directory with the specified directory name (if not already exist)

In [ ]:
from dotenv import set_key, get_key, find_dotenv
envPath = find_dotenv(raise_error_if_not_found=True)

mlSolutionPath = get_key(envPath, "mlSolutionPath")

In [ ]:
%%bash
# For visualizations, we prefer to use matplotlib library, so installing it
if python3 -c 'import pkgutil; exit(not pkgutil.find_loader("matplotlib"))';
then
	echo "matplotlib package found. Skipping re-installation...";
else
	echo "matplotlib package not found. Installing...";
    pip3 install matplotlib;
fi

### 8.3. Test scoring function and show result

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import urllib.request
import ssl
import requests
import json
import io
import os

def urlToImage(url):
    url = url.replace(os.path.sep, '/')
    resp = urllib.request.urlopen(url, context=ssl._create_unverified_context())
    image = bytearray(resp.read())
    return image

sampleImageURI = "https://live.staticflickr.com/3722/13741438294_401e031949_z.jpg"
sampleImageURI = "http://www.globaltimes.cn/Portals/0/attachment/2018/2018-12-04/0f255c7b-ab89-47ca-bbc8-917aa8fb767b.jpeg"

scoringURI = "http://127.0.0.1:5001/score"

imgData = urlToImage(sampleImageURI)
headers = {'Content-Type': 'image/jpeg'}
apiResponse = requests.post(scoringURI, headers=headers, data=imgData)

print("Raw API response: {0}".format(apiResponse.text))

# Parse resulting JSon output
jsonResult = json.loads(apiResponse.text)

print("Status: {0}".format(jsonResult["status"]))
print("Inference Time: {0} ms".format(jsonResult["time"]))
print("Detected object count: {0}".format(jsonResult["object_count"]))

plt.figure()
fig, ax = plt.subplots(1, figsize=(12,9))
ax.imshow(Image.open(io.BytesIO(imgData)))

statusCode = int(jsonResult["status"])
if statusCode > 0:
    print("Status Code: {0}".format(statusCode))
    exit

jsonResult = jsonResult["result"]
print("Detected objects:")
for k in jsonResult:
    print(k, "->", jsonResult[k])
    detRes = jsonResult[k]

    xmin = float(detRes["xmin"])
    ymin = float(detRes["ymin"])
    xmax = float(detRes["xmax"])
    ymax = float(detRes["ymax"])
    labelId = str(detRes["label"])
    confidence = detRes["confidence"]

    color = "blue" #(min(labelId * 12.5, 255), min(labelId * 7, 255), min(labelId * 5, 255))
    box_h = (ymax - ymin)
    box_w = (xmax - xmin)
    bbox = patches.Rectangle((xmin, ymin), box_w, box_h, linewidth=2, edgecolor=color, facecolor='none')
    ax.add_patch(bbox)
    plt.text(xmin, ymin, s=str(labelId), color='white', verticalalignment='top', bbox={'color': color, 'pad': 0})

plt.axis('off')
plt.show()